In [ ]:
//init cell

//common functions

const os = require('os');
const fs = require('fs');
const osc = require('osc');
const net = require('net');

function sliceStartStop(loop) {
    return loop.slice(1, -1)
}

function sortByTs(loop) {
    return loop.sort((e1, e2) => e1.ts-e2.ts);
}

const defaultGestureData = { //loopKey must ALWAYS be manually provided
    pos: "default",
    duration: 5,
    group: "default",
    key: "default",
    deltaLoop: true,
    deltaAccumulate: true,
    rotation: 0
};

function sendConfig(conf) {
    conf = Array.isArray(conf) ? conf : [conf];
    conf = conf.map(cnf => ({...defaultGestureData, ...cnf}));
    // udp_osc_port.send({address: "/launchConfig", args: oscArgs(JSON.stringify(conf))});
    const jsonStr = JSON.stringify({type: "launchConfig", data: conf});
    sockets.forEach(sock => sock.write(jsonStr + '\n' ));
}

function normalizeTime(loop) {
    const start_time = loop[0].ts;
    const dur = loop.slice(-1)[0].ts - start_time;
    touch_pts = loop.map(({ts, pos}) => ({pos, ts: (ts-start_time)/dur}));
    return touch_pts;
}

function deduplicate(loop) { //for fixing touchOSC bug of "lagging Y" redundant pre-message
    let filteredLoop = loop.filter((elem, i, arr) => i%2 != 0); //throw out even indices
    return filteredLoop;
    //todo - this only works when messages are guaranteed ordered (local UDP, tcp)
    //for non-ordered messages, probably most robust is to just toss first couple values to
    //get rid of "cut start" from redundant message (or to move cursor near intened start before recording)
}

function flipY(loop) {
    return loop.map(({ts, pos: {x, y}}) => ({ts, pos: {x, y: 1-y}}))
}

function perfectLoop(cleanedLoop) {
    let startPt = cleanedLoop[0];
    let endPt = cleanedLoop.slice(-1)[0];
    let netDelta = {x: endPt.pos.x - startPt.pos.x, y: endPt.pos.y - startPt.pos.y};
    let numDelt = cleanedLoop.length - 1
    let delta = {x: netDelta.x/numDelt, y: netDelta.y/numDelt};
    let closedLoop = cleanedLoop.map(({ts, pos: {x, y}}, i)=> ({ts, pos: {x: x-i*delta.x, y: y-i*delta.y}}));
    return closedLoop;
}

function oscArgs(...args) {
    let output  = args.map(e => {
        let type_str = typeof e;
        switch(type_str) {
            case 'string':
                return {type: 's', value: e}
            case 'number': 
                return {type: 'f', value: e}
            case 'boolean':
                return {type: 'i', value: e+0}
        }
    });
    return output;
}

function saveLoops(fileName) {
    if(!fileName) fileName = Date().toString().replaceAll(/[\: \)\(]/g, "-") + ".json";
    fs.writeFileSync(fileName, JSON.stringify({rawLoops, processedLoops}));
}
function loadLoopFile(fileName) {
    return JSON.parse(fs.readFileSync(fileName));
}
function updateLoops() {
    const dataString = JSON.stringify({type: "processedLoops", data: processedLoops});
    sockets.forEach(sock => sock.write( dataString + '\n' ));
}
function loadLoops(fileName) {
    let loops = loadLoopFile(fileName);
    rawLoops = loops.rawLoops;
    processedLoops = loops.processedLoops;
    updateLoops();
    return new Promise((resolve, reject) => setTimeout(resolve, 150));
}


//TCP server to OF

const tcp_port = 11999;
const tcp_host = '127.0.0.1';

const server = net.createServer();
server.listen(tcp_port, tcp_host, () => {
    console.log('TCP Server is running on port ' + tcp_port + '.');
});

const sockets = [];

server.on('connection', function(sock) {
    console.log('CONNECTED: ' + sock.remoteAddress + ':' + sock.remotePort);
    sockets.push(sock);

    const dataString = JSON.stringify({type: "processedLoops", data: (() => processedLoops)()});
    sock.write( dataString + '\n' );

    sock.on('data', function(data) {
        console.log('DATA ' + sock.remoteAddress + ': ' + data);
        // Write the data back to all the connected, the client will receive it as data from the server
        sockets.forEach(function(sock, index, array) {
            sock.write(sock.remoteAddress + ':' + sock.remotePort + " said " + data + '\n');
        });
    });

    // Add a 'close' event handler to this instance of socket
    sock.on('close', function(data) {
        let index = sockets.findIndex(function(o) {
            return o.remoteAddress === sock.remoteAddress && o.remotePort === sock.remotePort;
        })
        if (index !== -1) sockets.splice(index, 1);
        console.log('CLOSED: ' + sock.remoteAddress + ' ' + sock.remotePort);
    });
});

//OSC Server to touchOSC

const TCP_OSC_PORT = 7071;
const TCP_OSC_HOST = '192.168.7.246';

const tcp_osc_port = new osc.TCPSocketPort({});
// change to remote host/port (address of phone/tablet) - set touchOSC on device to TCPserver
tcp_osc_port.open(TCP_OSC_HOST, TCP_OSC_PORT); 

tcp_osc_port.on('ready', () => {
  console.log('ready OSC TCP');
});


const UDP_OSC_PORT = 7071;
const UDP_OSC_HOST = '0.0.0.0';

// Bind to a UDP socket to listen for incoming OSC events.
const udp_osc_port = new osc.UDPPort({
    localAddress: UDP_OSC_HOST,
    localPort: UDP_OSC_PORT,
    remoteAddress: '127.0.0.1',
    remotePort: 7072
});

udp_osc_port.on("ready", () => {
    console.log('ready OSC UDP');
});

udp_osc_port.open();

function clearAll() {
    udp_osc_port.send({address: "/clearAll", args: []});
}

let osc_port = tcp_osc_port;

let rawLoops = {};
let processedLoops = {};
let activeLoop = 'firstLoop';
let isRecording = false;
let isTouching = false;
const handlers = {};

function processLoop(loopKey) {
    processedLoops[loopKey] = flipY(normalizeTime(deduplicate(sortByTs(sliceStartStop(rawLoops[loopKey])))));
}

// s_ts is server timestamp, ts is client timestamp
handlers['/recording'] = (isRec) => {
    console.log("isRec", isRec);
    isRecording = isRec;
    if(isRecording) {
        rawLoops[activeLoop] = [{s_ts: Date.now(), pos: 'start'}];
    } else {
        rawLoops[activeLoop].push({s_ts: Date.now(), pos: 'end'});
        processLoop(activeLoop);
        updateLoops();
    }
}
const f = float => ({type: 'f', value: float}); //helper for formatting floats for OSC
handlers['/pos'] = (x, y, ts) => {
    if(isRecording) {
        rawLoops[activeLoop].push({s_ts: Date.now(), pos: {x, y}, ts});
    }
    udp_osc_port.send({address: '/touchPos', args: [f(x), f(y)]});
}
handlers['/pos/touch'] = (touching) => {
    isTouching = touching;
    udp_osc_port.send({address: '/touching', args: [f(isTouching)]});
}

let logAddresses = [];
let messageHandler = (message, timetag, info) => {
    let {address, args} = message;
    if(logAddresses.includes(address) || logAddresses.includes("all")) console.log(message);
    if(handlers[address]) handlers[address](...args); 
}
osc_port.on('message', messageHandler);

let loadFile = "test_loops.json";
if(loadFile) {
    console.log("loaded file", loadFile);
    let loops = loadLoopFile(loadFile);
    processedLoops = loops.processedLoops;
    rawLoops = loops.rawLoops;
}

"Setup Done"

In [ ]:
processedLoops

In [ ]:
// loadLoops('test_loops.json')
grid = 5
config = []
for(let i = 0; i < grid; i++) {
    for(let j = 0; j < grid; j++) {
        let pos = {x: i / grid, y: j / grid};
        let loopKey = "loop1";
        config.push({pos, loopKey, duration: 30})
    }
}
sendConfig(config);

In [ ]:
// tight descending spirals
// udp_osc_port.send({address: "/gridSize", args: oscArgs(4)});
// udp_osc_port.send({address: "/useVoronoi", args: oscArgs(1)});
// for(var i = 0; i < 100; i++) {
//     launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop1', 10, false, 'grp1', 'key'+i)});
//     setTimeout(launch, i*50)
// }

//wide+short descending spirals
// udp_osc_port.send({address: "/gridSize", args: oscArgs(10)});
for(var i = 0; i < 100; i++) {
    let speed = 10 + i/100;
    launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop1', speed, false, 'grp1', 'key'+i)});
    setTimeout(launch, i*10)
}

5

In [ ]:
conf = [];
for(var i = 0; i < 10; i++) {
    conf.push({loopKey: 'loop2', rotation: i/10 * Math.PI * 2, pos: {x: 0.5, y: 0.5}})
}
sendConfig(conf)

In [ ]:
clearAll()

In [ ]:
//sketches
async function sketch1() {
    await loadLoops('test_loops.json')
    let dev = 500; //40,
    for(var i = 0; i < dev; i++) {
        let dur = 15 + i/dev
        udp_osc_port.send({address: "/launch", args: oscArgs('loop1', dur, false, 'grp1', 'key'+i)});
    }
}

async function sketch2() {
    await loadLoops('test_loops.json')
    let launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop1', 30, false, 'grp1', 'key1')});
    for(var i = 0; i < 100; i ++) {
        setTimeout(launch, i*10)
        setTimeout(launch, 4000 + i*10)
    }
}

async function sketch3() {
    udp_osc_port.send({address: "/gridSize", args: oscArgs(4)});
    udp_osc_port.send({address: "/useVoronoi", args: oscArgs(1)});
    await loadLoops('test_loops.json')
    let launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop1', 30, false, 'grp1', 'key1')});
    for(var i = 0; i < 10; i ++) {
        setTimeout(launch, i*100)
    }
}

async function sketch4() {
    udp_osc_port.send({address: "/gridSize", args: oscArgs(4)});
    udp_osc_port.send({address: "/useVoronoi", args: oscArgs(1)});
    await loadLoops('test_loops.json')
    let launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop1', 60, false, 'grp1', 'key1')});
    for(var i = 0; i < 10; i ++) {
        setTimeout(launch, i*100)
        setTimeout(launch, 4000 + i*100)
    }
}

In [ ]:
//basic ws template

const ws = require('ws')

const wss = new ws.Server({ port: 8080 });

wss.on('connection', function connection(ws) {
  console.log("connected")
  ws.on('message', function message(data) {
    console.log('received: %s', data);
  });
});